In [2]:
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np

torch.manual_seed(73)

train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST('data', train=False, download=True, transform=transforms.ToTensor())

batch_size = 64

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

class ConvNet(torch.nn.Module):
    def __init__(self, hidden=64, output=10):
        super(ConvNet, self).__init__()        
        self.conv1 = torch.nn.Conv2d(1, 4, kernel_size=7, padding=0, stride=3)
        self.fc1 = torch.nn.Linear(256, hidden)
        self.fc2 = torch.nn.Linear(hidden, output)

    def forward(self, x):
        x = self.conv1(x)
        
        # the model uses the square activation function
        x = x * x
        # flattening while keeping the batch axis
        x = x.view(-1, 256)
        x = self.fc1(x)
        x = x * x
        x = self.fc2(x)
        
        return x


def train(model, train_loader, criterion, optimizer, n_epochs=10):
    # model in training mode
    model.train()
    for epoch in range(1, n_epochs+1):

        train_loss = 0.0
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # calculate average losses
        train_loss = train_loss / len(train_loader)

        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))
    
    # model in evaluation mode
    model.eval()
    return model


model = ConvNet()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model = train(model, train_loader, criterion, optimizer, 10)

Epoch: 1 	Training Loss: 0.420179
Epoch: 2 	Training Loss: 0.143187
Epoch: 3 	Training Loss: 0.095321
Epoch: 4 	Training Loss: 0.073644
Epoch: 5 	Training Loss: 0.062036
Epoch: 6 	Training Loss: 0.050439
Epoch: 7 	Training Loss: 0.045990
Epoch: 8 	Training Loss: 0.039238
Epoch: 9 	Training Loss: 0.036038
Epoch: 10 	Training Loss: 0.033039


In [3]:
def test(model, test_loader, criterion):
    # initialize lists to monitor test loss and accuracy
    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    # model in evaluation mode
    model.eval()

    for data, target in test_loader:
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item()
        # convert output probabilities to predicted class
        
        _, pred = torch.max(output, 1)
        # compare predictions to true label
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        # calculate test accuracy for each object class
        for i in range(len(target)):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    # calculate and print avg test loss
    test_loss = test_loss/len(test_loader)
    print(f'Test Loss: {test_loss:.6f}\n')

    for label in range(10):
        print(
            f'Test Accuracy of {label}: {int(100 * class_correct[label] / class_total[label])}% '
            f'({int(np.sum(class_correct[label]))}/{int(np.sum(class_total[label]))})'
        )

    print(
        f'\nTest Accuracy (Overall): {int(100 * np.sum(class_correct) / np.sum(class_total))}% ' 
        f'({int(np.sum(class_correct))}/{int(np.sum(class_total))})'
    )
    
test(model, test_loader, criterion)

Test Loss: 0.112927

Test Accuracy of 0: 99% (972/980)
Test Accuracy of 1: 99% (1128/1135)
Test Accuracy of 2: 98% (1019/1032)
Test Accuracy of 3: 98% (999/1010)
Test Accuracy of 4: 97% (960/982)
Test Accuracy of 5: 96% (858/892)
Test Accuracy of 6: 97% (933/958)
Test Accuracy of 7: 95% (985/1028)
Test Accuracy of 8: 97% (945/974)
Test Accuracy of 9: 98% (991/1009)

Test Accuracy (Overall): 97% (9790/10000)


In [4]:
# conv1
for i, f in enumerate(model.conv1.weight.detach().numpy()):
    np.savetxt(f"../model/CW{i}.csv", f[0], delimiter=",")
np.savetxt(f"../model/CB.csv", model.conv1.bias.detach().numpy(), delimiter=",")

# LC1
np.savetxt(f"../model/LW1.csv", model.fc1.weight.detach().numpy(), delimiter=",")
np.savetxt(f"../model/LB1.csv", model.fc1.bias.detach().numpy(), delimiter=",")

# LC2
np.savetxt(f"../model/LW2.csv", model.fc2.weight.detach().numpy(), delimiter=",")
np.savetxt(f"../model/LB2.csv", model.fc2.bias.detach().numpy(), delimiter=",")

In [7]:
import csv 

with open("../mnist_test.csv") as f:
    line = f.readline().strip().split(",")
    label = line[0]
    img = np.array([float(x) / 255 for x in line[1:]], dtype=np.float32).reshape((28, 28))

img = torch.tensor([img])
model.conv1(img)

/var/folders/14/xy1pd25s1j3b39t4lkp1cc4r0000gn/T/ipykernel_3744/63792817.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)
  img = torch.tensor([img])


tensor([[[ 3.9283e-01,  3.9283e-01,  3.9283e-01,  3.9283e-01,  3.9283e-01,
           3.9283e-01,  3.9283e-01,  3.9283e-01],
         [ 2.3382e-01, -2.7359e-01,  4.7459e-01,  3.7585e-01,  2.9909e-01,
           4.2801e-01,  8.8767e-01,  3.0873e-01],
         [ 2.6895e-02,  2.6429e-01,  8.3365e-02,  1.9869e-01,  1.0552e-01,
           4.2180e-03,  1.5783e+00,  2.2101e-01],
         [ 3.4234e-01,  3.2934e-01,  1.8478e-01,  2.3201e-01, -9.3780e-01,
           1.2003e+00,  1.7609e-01,  3.2475e-01],
         [ 3.9283e-01,  3.9283e-01,  3.9283e-01,  1.5059e-01, -1.1325e+00,
           2.2095e+00, -3.0141e-01,  3.9283e-01],
         [ 3.9283e-01,  3.9283e-01,  3.6778e-01, -9.5790e-01,  1.8679e+00,
           1.8811e-01,  3.2118e-01,  3.9283e-01],
         [ 3.9283e-01,  3.9283e-01, -2.6800e-01, -2.4297e-01,  1.6880e+00,
          -1.2719e-01,  3.9283e-01,  3.9283e-01],
         [ 3.9283e-01,  3.9283e-01, -1.1077e+00,  2.1386e+00, -1.4975e-02,
           3.6795e-01,  3.9283e-01,  3.9283e-01]],